# Predicting Labels using Alpaca

In [1]:
# Install all the required packages.
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip uninstall peft -y
!pip install -q git+https://github.com/huggingface/peft.git@0769587a3cd80ad2ae508cc06efbf54ddca821b3

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Found existing installation: peft 0.4.0
Uninstalling peft-0.4.0:
  Successfully uninstalled peft-0.4.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# Import all needed libreries
import pandas as pd
import numpy as np
import tqdm
import torch

In [3]:
#The following code loads the Alpaca LoRA pretrained model.
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("linhvu/decapoda-research-llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained("linhvu/decapoda-research-llama-7b-hf", load_in_8bit=True, device_map="auto")

model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'LLaMATokenizer'.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [5]:
#Evaluate function that we will use to call Alpaca LoRA.
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
    repetition_penalty=1.03
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    response = []
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        response.append(output.split("### Response:")[1].strip())
    return response

In [7]:
#read and prepare dataset
df = pd.read_pickle('./data_turns.pkl')

df['turn_len'] = df.turn_text.apply(lambda x: len(x['text'].split()))
df['turn_text'] = df.apply(lambda row: {
                                            'author': row['turn_text']['author'], 
                                            'text' : row['topic'].replace('_', ' ') + ' [SEP] ' +  row['turn_text']['text']
                                           } ,axis=1)
df = df.groupby('task_id').agg({'turn_text': lambda rows: list(rows),
                                'topic': lambda rows: list(rows)[0],
                                'topic_func_label': lambda rows: list(rows),
                                'dlg_act_label': lambda rows: list(rows),
                                'exp_act_label': lambda rows: list(rows)}).reset_index()
df['num_turns_per_topic'] = df.topic_func_label.apply(lambda x : len(x))

In [ ]:
#Split the data in training set 60%, validation set 20%, test set 20%
_, valid, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

valid_df = valid.copy()

In [10]:
#test the prompt to predict the explanation move label
INPUT = '''
  {'turn': 1 ,'author': 'Explainer', 'text': "You know, what we do in our studies, is we actually track people, for say a week, and they have to get in within this range of amount of sleep and then it has to be both amount of sleep and how stable it is. That variability can kind of impact it and certainly might have an impact on the machinery that's happening in the brain.", "label": "(E03) Provide an explanation"},
  {'turn': 2 ,'author': 'Explainee', 'text': "I often think that people, many people think that they can get by on very little sleep where the less than average amount of sleep, from what I know from the research, you know if you actually test those people on different batteries of cognitive tasks, you often find that their performance is slightly worse than they had anticipated.", "label": "(E03) Provide an explanation"},
  {'turn': 3 ,'author': 'Explainer', 'text': 'Yeah. So what did you learn about sleep today?', "label": "(E01) Testing understanding"},
  {'turn': 4 ,'author': 'Explainee', 'text': "I guess I learned a system that is, you know, very heavily regulated and influenced by some of these older conserved sub-cortical systems in the brain, and that there's, as you called it, like a flip flop function between wakefulness and sleep that's regulated by these specific dopaminergic, neurogenergic, like neurotransmitters, that manifest in different sub-cortical parts of the brain.", "label": "(E03) Provide an explanation"}'''

EXPLANATION_MOVE_PROMPT = '''
Given a sequence of dialogue turns between an 'Explainer' and an 'Explainee', each turn is accompanied by a text and a corresponding label. These labels categorize the nature of the dialogue turn. The 10 distinct labels are:

(E01) Testing understanding: Checking whether the listener understood what was being explained;
(E02) Testing prior knowledge: Checking the listener’s prior knowledge of the turn’s topic;
(E03) Provide an explanation: explaining any concept or a topic to the listener;
(E04) Ask for an explanation: Requesting any explanation from the listener;
(E05) Signaling understanding: Informing the listener that their last utterance was understood;
(E06) Signal non-understanding: Informing the listener that the utterance was not understood;
(E07) Providing feedback: Responding qualitatively to an utterance by correcting errors or similar;
(E08) Providing Assessment: Assessing the listener by rephrasing their utterance or giving a hint;
(E09) Introducing Extraneous Information Giving additional information to foster a complete understanding;
(E10) Other: Making any other explanation move.

Based on the provided dialogue turns and their respective labels, predict only the label of the next turn in the dialogue!!!'''

response = evaluate(EXPLANATION_MOVE_PROMPT, INPUT)
response

['(E04) Ask for an explanation']

In [11]:
#function to test one dialogue on explanation move dimention
#return how many correct and incorrect labels was predicted 
def test_one_dig(dig):
    correct, incorrect = 0, 0
    INPUT = ""
    for i, (turn_text, label) in enumerate(zip(dig["turn_text"][:-1], dig["exp_act_label"][:-1])):
        if i == 10 : break
        INPUT += f"turn number: {i+1}, author: {turn_text['author']}, turn text: {turn_text['text']}, label: {label}.\n"

        response = evaluate(EXPLANATION_MOVE_PROMPT, INPUT)
        print(response)
        
        try:
            if response[0][2:4] == dig["exp_act_label"][i+1][2:4]:
                correct += 1
            else:
                incorrect += 1
        except:
            
            print("Unexpected response: ", response[0])
            incorrect += 1
            
    return correct, incorrect

In [12]:
#run the experiment for predicting explanation move labels on validset
from tqdm import tqdm

results = []
correct, incorrect = 0, 0
for i in tqdm(range(92)):
    print("New dig:", i, "\tTask id", valid_df.iloc[i]['task_id'])
    result_dig = test_one_dig(valid_df.iloc[i])
    results.append((result_dig[0], result_dig[1]))
    correct += result_dig[0]
    incorrect += result_dig[1]
    print("Correct", result_dig[0], ', Incorrect:', result_dig[1])
    print("Overall correct", correct, ', Overall incorrect:', incorrect)
    print("F1 Score:", correct / (correct + incorrect))

  0%|          | 0/92 [00:00<?, ?it/s]

New dig: 0 	Task id 28641
['(E05) Signaling understanding.']
['(E04) Ask for an explanation.']
['(E04) Ask for an explanation.']
['(E04) Ask for an explanation.']
['(E07) Providing Feedback.']


  1%|          | 1/92 [00:45<1:08:52, 45.42s/it]

['(E07) Providing Feedback.']
Correct 3 , Incorrect: 3
Overall correct 3 , Overall incorrect: 3
F1 Score: 0.5
New dig: 1 	Task id 28454
['(E05) Signaling understanding: Informing the listener that their last utterance was understood.']
['(E10) Other: Making any other explanation move.']
['(E10) Other: Making any other explanation move.']
['(E09) Introducing Extraneous Information Giving additional information to foster a complete understanding.']
['(E10) Other']
['label: (E03) Provide an explanation.']
['(E03) Provide an explanation.']


  2%|▏         | 2/92 [03:23<2:47:53, 111.92s/it]

...

['(E07) Providing Feedback.']
Correct 3 , Incorrect: 3
Overall correct 102 , Overall incorrect: 556
F1 Score: 0.15501519756838905
New dig: 90 	Task id 28819
['(E04) Ask for an explanation.']
['(E04) Ask for an explanation.']
['(E03) Provide an explanation.']
['(E04) Ask for an explanation.']


 99%|█████████▉| 91/92 [2:07:43<01:10, 70.83s/it]

['(E07) Providing Feedback.']
Correct 2 , Incorrect: 3
Overall correct 104 , Overall incorrect: 559
F1 Score: 0.1568627450980392
New dig: 91 	Task id 28811
['(E05) Signaling understanding.']
['(E04) Ask for an explanation.']
['(E04) Ask for an explanation.']
['Based on the provided dialogue turns and their respective labels, the next turn in the dialogue is (E04) Ask for an explanation.']


100%|██████████| 92/92 [2:09:24<00:00, 84.40s/it]

['Based on the provided dialogue turns and their respective labels, predict only the label of the next turn in the dialogue.']
Correct 1 , Incorrect: 4
Overall correct 105 , Overall incorrect: 563
F1 Score: 0.15718562874251496


In [17]:
#test the prompt to predict the topic label
INPUT = '''
  {'turn': 1 ,'author': 'Explainer', 'text': "You know, what we do in our studies, is we actually track people, for say a week, and they have to get in within this range of amount of sleep and then it has to be both amount of sleep and how stable it is. That variability can kind of impact it and certainly might have an impact on the machinery that's happening in the brain.", "label": "(T03) A related topic"},
  {'turn': 2 ,'author': 'Explainee', 'text': "I often think that people, many people think that they can get by on very little sleep where the less than average amount of sleep, from what I know from the research, you know if you actually test those people on different batteries of cognitive tasks, you often find that their performance is slightly worse than they had anticipated.", "label": "(T01) It is the main topic"},
  {'turn': 3 ,'author': 'Explainer', 'text': 'Yeah. So what did you learn about sleep today?', "label": "(T01) It is the main topic"},
  {'turn': 4 ,'author': 'Explainee', 'text': "I guess I learned a system that is, you know, very heavily regulated and influenced by some of these older conserved sub-cortical systems in the brain, and that there's, as you called it, like a flip flop function between wakefulness and sleep that's regulated by these specific dopaminergic, neurogenergic, like neurotransmitters, that manifest in different sub-cortical parts of the brain.", "label": "(T03) A related topic"}'''

TOPIC_PROMPT = '''
Given a sequence of dialogue turns between an 'Explainer' and an 'Explainee', each turn is accompanied by a text and a corresponding label. These labels categorize the nature of the dialogue turn. The 4 distinct labels are:

(T01) It is the main topic;
(T02) A subtopic;
(T03) A related topic;
(T04) Other - No topic was introduced;

Based on the provided dialogue turns and their respective labels, predict only the label of the next turn in the dialogue!!!'''

response = evaluate(TOPIC_PROMPT, INPUT)
response

['(T04) Other - No topic was introduced']

In [18]:
#function to test one dialogue on topic dimention
#return how many correct and incorrect labels was predicted 
def test_one_dig_topic(dig):
    correct, incorrect = 0, 0
    INPUT = ""
    for i, (turn_text, label) in enumerate(zip(dig["turn_text"][:-1], dig["topic_func_label"][:-1])):
        if i == 10 : break
            
        INPUT += f"turn number: {i+1}, author: {turn_text['author']}, turn text: {turn_text['text']}, label: {label}.\n"
       
        response = evaluate(TOPIC_PROMPT, INPUT)
        print(response)
        
        try:
            if response[0][2:4] == dig["topic_func_label"][i+1][2:4]:
                correct += 1
            else:
                incorrect += 1
        except:
            print("Unexpected response: ", response[0])
            incorrect += 1
            
    return correct, incorrect

In [19]:
#run the experiment for predicting topic labels validset
results_topic = []
correct, incorrect = 0, 0
for i in range(92):
    print("New dig:", i, "\tTask id", valid_df.iloc[i]['task_id'])
    result_dig = test_one_dig_topic(valid_df.iloc[i])
    results_topic.append((result_dig[0], result_dig[1]))
    correct += result_dig[0]
    incorrect += result_dig[1]
    print("Correct", result_dig[0], ', Incorrect:', result_dig[1])
    print("Overall correct", correct, ', Overall incorrect:', incorrect)
    print("F1 Score:", correct / (correct + incorrect))

New dig: 0 	Task id 28641
['(T02) A subtopic.']
['(T03) A related topic.']
['(T03) A related topic.']
['(T04) Other - No topic was introduced.']
['(T02) A subtopic.']
['(T02) A subtopic.']
Correct 2 , Incorrect: 4
Overall correct 2 , Overall incorrect: 4
F1 Score: 0.3333333333333333
New dig: 1 	Task id 28454
['(T02) A subtopic']
['(T02) A subtopic']
['(T04) Other - No topic was introduced.']
['(T04) Other - No topic was introduced.']
['(T04) Other - No topic was introduced.']
['label: (T04) Other - No topic was introduced.']
['label: (T04) Other - No topic was introduced.']
['(T01) It is the main topic.']
Correct 3 , Incorrect: 5
Overall correct 5 , Overall incorrect: 9
F1 Score: 0.35714285714285715
New dig: 2 	Task id 28407
['(T02) A subtopic.']
['(T02) A subtopic.']
['(T04) Other - No topic was introduced.']
['(T04) Other - No topic was introduced.']
['(T04) Other - No topic was introduced.']
['(T03) A related topic.']
['(T03) A related topic.']
['(T03) A related topic.']
['(T03) A r